In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain-community
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install tiktoken
!pip install langchain-google-genai
!pip install chromadb
!pip install langchain_huggingface
!pip install ragas
!pip install faiss-cpu
!pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [3]:
import os
import re
import time
import pickle
import requests
import json
import pandas as pd
import bs4
import tiktoken
import getpass
from tqdm import tqdm
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import clear_output

from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from langchain.vectorstores import Milvus
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate

from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

from langchain_huggingface.embeddings import HuggingFaceEmbeddings  # 원래 사용하던 허깅페이스 (참고용)

### API 환경설정

In [6]:
# NCP API Key - 클로바X & bgm-3 임베딩 모델

# 환경변수 등록
os.environ["NCP_CLOVASTUDIO_API_KEY"] = getpass.getpass("Test Key 입력: ")

# url
os.environ["NCP_CLOVASTUDIO_API_URL"] = "https://clovastudio.stream.ntruss.com/"

# CLOVA Studio 임베딩(v2) 도구와 채팅 모델 인스턴스 생성
embeddings = ClovaXEmbeddings(model="bge-m3")
chat = ChatClovaX(model="HCX-003")

Test Key 입력: ··········


### Test

In [63]:
# print(os.environ.get("NCP_CLOVASTUDIO_API_KEY"))

In [8]:
# # 임베딩 test
# sample_text = "임베딩을 테스트합니다."
# embedding_vector = embeddings.embed_query(sample_text)
# print("임베딩 결과:", embedding_vector)

임베딩 결과: [-1.3408203, 0.36645508, -0.49267578, 0.33422852, -0.10406494, 0.25634766, 1.5878906, -0.030075073, 0.6894531, 0.4543457, 0.13208008, -0.7128906, 0.9711914, -0.20043945, 0.35229492, -0.16967773, 0.14282227, 0.04458618, 0.028961182, -1.0234375, -0.5053711, -1.1533203, 0.75, -0.39257812, 0.24536133, 0.85546875, -0.079956055, 0.093444824, 0.8383789, -0.33569336, 0.80371094, -0.44995117, 0.7236328, -1.5986328, -0.97558594, -1.1503906, 0.70654297, -0.25390625, -0.90185547, 0.91015625, -0.1529541, 0.11975098, -0.0368042, -0.7685547, 0.1303711, -0.72314453, 0.25878906, -0.8535156, -1.1962891, 0.94091797, 0.9135742, 0.59472656, -0.16052246, -0.15234375, 0.022872925, 0.6977539, -0.6113281, 0.14318848, -1.7900391, -0.2939453, -0.83251953, 1.2011719, -0.81884766, 0.39624023, 0.5566406, 2.171875, -0.39794922, 0.4885254, -0.91503906, -1.4980469, -0.47827148, 1.0332031, -0.5175781, 0.27319336, -2.0332031, 1.0009766, 0.1977539, -0.94140625, -0.7470703, 0.69140625, 0.6713867, -0.90478516, 0.66

In [9]:
# messages = [
#     (
#         "system",
#         "CLOVA Studio는 HyperCLOVA X 언어 모델을 활용하여 AI 서비스를 손쉽게 만들 수 있는 개발 도구입니다.",
#     ),
#     ("human", "CLOVA Studio가 무엇인가요?"),
# ]
# ai_msg = chat.invoke(messages)
# ai_msg

AIMessage(content='CLOVA Studio는 네이버에서 개발한 인공지능(AI) 기술인 HyperCLOVA X를 기반으로 한 노 코드(No Code) AI 개발 플랫폼입니다. \n\n사용자는 CLOVA Studio를 통해 복잡한 코딩 없이도 자연어 처리, 이미지 인식, 음성 인식 등 다양한 AI 기능을 구현할 수 있습니다. 또한, API 형태로 제공되는 다양한 데이터와 연동하여 더욱 풍부한 AI 서비스를 개발할 수 있습니다.\n\nCLOVA Studio는 다음과 같은', additional_kwargs={}, response_metadata={'stop_reason': 'length', 'input_length': 34, 'output_length': 101, 'seed': 395853271, 'ai_filter': None}, id='run-37f60ffb-3e22-4681-8e6a-215c79810f23-0', usage_metadata={'input_tokens': 34, 'output_tokens': 101, 'total_tokens': 135})

### 임베딩

### 데이터 (sample : 10000개)

In [13]:
df = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/sampled_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10000 non-null  int64 
 1   ISBN        10000 non-null  int64 
 2   분류          9852 non-null   object
 3   제목          10000 non-null  object
 4   부제          6686 non-null   object
 5   저자          10000 non-null  object
 6   발행자         10000 non-null  object
 7   발행일         10000 non-null  object
 8   페이지         10000 non-null  int64 
 9   가격          10000 non-null  int64 
 10  표지          10000 non-null  object
 11  책소개         9871 non-null   object
 12  저자소개        7576 non-null   object
 13  목차          9065 non-null   object
 14  출판사리뷰       5834 non-null   object
 15  추천사         106 non-null    object
dtypes: int64(4), object(12)
memory usage: 1.2+ MB


In [14]:
df = df.drop(columns=['Unnamed: 0'])
df = df.sample(n=2000, random_state=2025)

In [37]:
df.head(50)

,ISBN,분류,제목,부제,저자,발행자,발행일,페이지,가격,표지,책소개,저자소개,목차,출판사리뷰,추천사
6448,9791190631136,['국내도서 > 사회과학 > 사회문제 > 범죄문제'],연쇄범죄란 무엇인가,연쇄범죄에 대한 이해와 연쇄범죄 수사전문화,김복준 지음,우물이있는집,2020-11-19T15:00:00.000Z,232,15000,https://image.aladin.co.kr/product/25610/35/co...,"“범죄 없는 그날까지 달린다.” 연쇄범죄의 유형과 특성, 원인에 대한 분석은 기존의...",김복준 저자 : 김복준 1982년 경찰에 입문하여 2014년 동두천경찰서 수사과장으...,프롤로그 \n‘사회적 테러’인 연쇄범죄에 대한 대비가 필요한 시점이다\r\n \n0...,《연쇄범죄란 무엇인가》의 주요 내용과 구성 연쇄범죄에 대한 이해와 실제사건의 흥미로...,NaN
3544,9791163131076,['국내도서 > 수험서/자격증 > 공무원 수험서 > 소방공무원(승진) > 소방승진'...,소방승진 시험대비 The쉬운(기출) 소방전술 객관식,2022,조동훈 지음,캠버스,2022-04-28T15:00:00.000Z,421,32000,https://image.aladin.co.kr/product/29354/52/co...,안녕하십니까? 조동훈입니다. 저는 30년 가까운 세월 동안 대학과 학원에서 소방의 ...,"조동훈 소방전술, 소방법령 전술 전문 저자이다. 저서로는 『총 12년 기출집(법규)...",1편 화재분야(화재1)\n01 화재진압 및 현장활동··················...,안녕하십니까? 조동훈입니다. 저는 30년 가까운 세월 동안 대학과 학원에서 소방의 ...,NaN
3904,9791165864750,['국내도서 > 인문학 > 동양철학 > 유교철학/주역 > 유교철학 일반'],역주 의례주소 사혼례 2,NaN,정병섭 저 지음,학고방,2022-08-30T15:00:00.000Z,465,38000,https://image.aladin.co.kr/product/31496/29/co...,"이 문헌의 저작자는 확정할 수 없지만, 공자가 제자들에게 가르쳤던 예(禮)를 후학들...",NaN,NaN,NaN,NaN
9739,9791191757248,"['국내도서 > 고전 > 우리나라 옛글 > 산문', '국내도서 > 소설/시/희곡 >...","세 살 적 배움, 평생을 이끈다",사소절 동규편,"이덕무 지음, 김종권 옮김",명문당,2021-11-16T15:00:00.000Z,500,20000,https://image.aladin.co.kr/product/28348/0/cov...,사소절은 실학자인 아정 이덕무(1741~1793)의 저작이다. 남자의 예법을 다룬 ...,이덕무 영조 17년에 태어나 정조 17년까지 활약한 조선 후기 문장가이자 대표적인 ...,1. 행동거지[動止동지]\n童規－動止 1“어린이의 성장과 발전은 천천히 지속적으로 ...,NaN,NaN
8295,9788985817974,['국내도서 > 자기계발 > 시간관리/정보관리 > 시간관리'],인생을 바꾼 숨은 시간의 기적,숨은 시간을 찾아 더 많은 일을 하고 더 여유로운 삶을 누리는 시간관리법,김지원 지음,태인문화사,2021-12-29T15:00:00.000Z,264,15000,https://image.aladin.co.kr/product/28618/29/co...,"성공의 기본이자 출발점 시간관리법, 숨어 있는 시간을 찾아라! 누구에게나 하루에 주...","김지원 저자 : 김지원 대한민국 평범한 공무원으로 통일부, 방위사업청을 거쳐 현재 ...",프롤로그\nPart 1. 시간 찾기를 해야만 하는 단 한 가지 이유\n01. 인생을...,숨은 시간을 찾아 하루를 48시간처럼 활용하라! 하루 24시간은 누구에게나 공평하게...,NaN
3600,9791137208698,"['국내도서 > 어린이 > 초등5~6학년 > 동화/명작/고전', '국내도서 > 어린...",자에는 자로 '영어 소설 쓰기' (주니어-영어원서) : MEASURE FOR MEA...,NaN,셰익스피어 지음,BOOKK(부크크),2020-06-07T15:00:00.000Z,99,7400,https://image.aladin.co.kr/product/24361/70/co...,자에는 자로 '영어 소설 쓰기' (주니어-영어원서) MEASURE FOR MEASU...,셰익스피어 저자 : 셰익스피어 (WILLIAM SHAKESPEARE) WILLIAM...,자에는 자로 \r\n'영어 소설 쓰기' \r\n(주니어-영어원서)\r\n \nMEA...,NaN,NaN
1457,9791137288782,['국내도서 > 경제경영 > 재테크/투자 > 부동산/경매'],범어동갤러리 두번째이야기,NaN,박윤주 지음,BOOKK(부크크),2022-07-15T15:00:00.000Z,265,18300,https://image.aladin.co.kr/product/29875/37/co...,NaN,박윤주 범어동 갤러리는 현재 수성구 아파트를 분석하는 블로그를 운영중이며 직접 현장...,NaN,NaN,NaN
6938,9791190712415,['국내도서 > 대학교재/전문서적 > 경상계열 > 무역'],2020 KIET 방산수출 10대 유망국가,NaN,김미정 지음,KIET,2021-02-04T15:00:00.000Z,166,7000,https://image.aladin.co.kr/product/26361/64/co...,▶ 이 책은 KIET의 2020 KIET 방산수출 10대 유망국가를 다룬 정부간행물...,NaN,요약\n제1장 서론\n1. 연구 배경\n2. 연구 필요성\n3. 연구 목적\n4. ...,NaN,NaN
984,9791198126597,"['국내도서 > 에세이 > 한국에세이', '국내도서 > 에세이 > 사진/그림 에세이...",천공의 섬 아저씨 (큰글자도서),아제세이,정윤섭 저 지음,핌,2023-01-19T15:00:00.000Z,204,36000,https://image.aladin.co.kr/product/31041/18/co...,아재 (아저씨) + 에세이 = 아제세이 영화 [공공의 적] 시나리오 작가 정윤섭의 ...,저 : 정윤섭 방송PD를 시작으로 지금은 시나리오 작가로 활약 중이다. [SBS 출...,<b>1. 식사류</b><br/>오늘의 이모 / 친구에게 전화가 왔다 / 편의점 맥...,아재 (아저씨) + 에세이 = 아제세이 영화 〈공공의 적〉 시나리오 작가 정윤섭의 ...,NaN
9586,9791189204846,"['국내도서 > 외국어 > 영어회화 > 생활영어', '국내도서 > 외국어 > 영어회...",쓰면서 말해봐 영어회화 세트,"기본편, 여행편, 일상편",이서영 지음,랭컴,2021-03-31T15:00:00.000Z,432,30000,https://image.aladin.co.kr/product/26882/74/co...,"쓰면 무조건 회화가 된다! 이 책은 이렇게 완성되었다! 유창한 회화를 위해 기본편,...",이서영 저자 : 이서영 저자 이서영은 대학에서 영어영문학을 전공하였으며 현재는 영어...,『기본편』 \nPART 01 인사 표현 \n01일상적으로 인사할 때12\r\n02근...,회화는 쓰기가 답이다! 1. 적혀 있는 그대로 읽으면서 따라 쓴다 2. 원어민의 발...,NaN


In [25]:
# bge-m3 임베딩 모델
ncp_embeddings = ClovaXEmbeddings(model="bge-m3")

# HCX-003 모델 - llm
llm_clova = ChatClovaX(model="HCX-003")

In [16]:
def split_text(text, chunk_size=1000, overlap=100):
    if text is None or pd.isnull(text):
        return [""]
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

metadata_columns = ['ISBN', '페이지', '가격', '제목', '저자']
vector_doc_columns = ['제목', '분류', '저자','저자소개', '책소개', '목차', '출판사리뷰','추천사']

In [17]:
RAG_DB = []
for index, row in df.iterrows():
    doc_text = ""
    for col in vector_doc_columns:
        value = row.get(col, "")
        if pd.isnull(value):
            value = ""
        doc_text += f"{col} : {value}\n"
    chunks = split_text(doc_text)

    metadata = {}
    for col in metadata_columns:
        metadata[col] = row.get(col, None)

    for chunk in chunks:
        RAG_DB.append({
            'text': chunk,
            'metadata': metadata
        })

documents = [
    Document(
        page_content=entry['text'],
        metadata=entry['metadata']
    ) for entry in RAG_DB
]


In [21]:
embedding_file = "/content/drive/MyDrive/aiffel_final_project/data_renew/ncp_bge_m3_embeddings.pkl"

if os.path.exists(embedding_file):
    # 있으면 불러오기
    with open(embedding_file, "rb") as f:
        saved_data = pickle.load(f)
    all_text_embedding_pairs = saved_data["embeddings"] # 핵심
    all_metadata_list = saved_data["metadata"]          # 메타
    print("임베딩 데이터 불러오기")
else:
    # 없으면 새로 임베딩
    all_text_embedding_pairs = []
    all_metadata_list = []

    batch_size = 2000
    for i, doc in enumerate(tqdm(documents, desc="NCP 임베딩(bge-m3) 처리 중", unit="document")):
        text_chunk = doc.page_content
        embedding = None

        # 호출 규정 제한 걸리면 재시도 except
        while embedding is None:
            try:
                embedding = ncp_embeddings.embed_query(text_chunk)
            except Exception as e:
                print(f"[에러] 문서 {i} 임베딩 실패: {e}")
                # 429 : many requests error
                if "429" in str(e):
                    print(f"[경고] 요청 제한 초과. 대기 후, 재시도")
                    time.sleep(5)  # 5초 대기 후 재시도
                    continue
                embedding = None  # 기타 에러가 나면 None으로 설정

        all_text_embedding_pairs.append((text_chunk, embedding)) # 청크와 원문 튜플
        all_metadata_list.append(doc.metadata)                   # 메타데이터

        # 1건당 1초 딜레이
        time.sleep(0.5)

        # 2000건마다 20초 딜레이
        if (i + 1) % batch_size == 0:
            print("배치 처리 2000건 완료, 20초 대기 중...")
            time.sleep(5)

    # 저장
    with open(embedding_file, "wb") as f:
        pickle.dump({
            "embeddings": all_text_embedding_pairs,
            "metadata": all_metadata_list
        }, f)
    print("저장 완료")

NCP 임베딩(bge-m3) 처리 중:   0%|          | 0/6824 [00:00<?, ?document/s]

[에러] 문서 29 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 대기 후, 재시도
[에러] 문서 29 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 대기 후, 재시도
[에러] 문서 29 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 대기 후, 재시도
[에러] 문서 29 임베딩 실패: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}
[경고] 요청 제한 초과. 대기 후, 재시도
[에러] 문서 69 임베딩 실패: Error response 429 while 

In [24]:
vectorstore = FAISS.from_embeddings(
    text_embeddings=all_text_embedding_pairs,
    metadatas=all_metadata_list,
    embedding=ncp_embeddings
)

In [26]:
# RetrievalQA 체인 구성
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova,
    retriever=dense_retriever,
    return_source_documents=True
)

In [51]:
# 프롬프트
multi_turn_prompt = PromptTemplate(
    input_variables=["history", "query"],
    template="""
[대화 맥락]
사용자 대화 내역:
{history}
사용자의 마지막 질문: "{query}"

[역할]
너는 도서관 사서 챗봇이다. 사용자의 책 추천 요청을 분석하여, 반드시 아래 항목을 출력하라.

[출력 항목]
1. 검색 확률: 0에서 1 사이의 숫자 (예: 0.9)
2. 기본 검색 쿼리: 반드시 유효한 검색 쿼리 텍스트 (예: "한국 고전 소설")
3. 추가 질문: 필요한 경우 추가 정보 요청, 충분하면 빈 문자열

출력 예시:
검색 확률: 0.9
기본 검색 쿼리: "한국 고전 소설"
추가 질문: ""

[지시]
- 대화 내용을 10~20 단어로 요약하고, 위 항목들을 반드시 모두 출력하라.
- "기본 검색 쿼리"가 비어 있거나 "None"이면 안 된다.
"""
)
search_query_chain = LLMChain(llm=llm_clova, prompt=multi_turn_prompt)

final_query_prompt = PromptTemplate(
    input_variables=["history", "fallback"],
    template="""
[대화 요약]
{history}

[기본 검색 쿼리]
{fallback}

위 내용을 바탕으로, 사용자 선호도를 종합하여 DB 검색에 사용할 최종 검색 쿼리를 생성하라.
최종 출력은 반드시 아래 형식을 준수하라:
1. 검색 확률: {{score}}
2. 검색 쿼리: "{{final_search_query}}"
3. 추가 질문: "{{follow_up_question}}"
4. 추천 이유: "{{reason}}"

(추천 이유 정보가 부족하면 "{{reason}}"에는 "추천 이유 정보 없음"이라고 출력하라.)
최종 검색 쿼리:
"""
)
final_query_chain = LLMChain(llm=llm_clova, prompt=final_query_prompt)

In [52]:
def extract_field(text, field_name):
    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""

In [53]:
MIN_INFO_LENGTH = 10

def generate_answer(query):
    result = dpr_qa_chain.invoke(query)
    source_docs = result['source_documents']

    retrieved_isbns = set()
    for doc in source_docs:
        isbn = doc.metadata.get("ISBN")
        if isbn:
            retrieved_isbns.add(isbn)

    aggregated_docs = []
    for isbn in retrieved_isbns:
        book_docs = [doc for doc in documents if doc.metadata.get("ISBN") == isbn]
        if not book_docs:
            continue
        aggregated_text = "\n".join([doc.page_content for doc in book_docs])
        aggregated_docs.append(Document(page_content=aggregated_text, metadata=book_docs[0].metadata))

    formatted_answers = []
    for doc in aggregated_docs:
        metadata = doc.metadata
        title = metadata.get("제목") or extract_field(doc.page_content, "제목")
        author = metadata.get("저자") or extract_field(doc.page_content, "저자")
        publisher_review = extract_field(doc.page_content, "출판사리뷰")
        book_intro = extract_field(doc.page_content, "책소개")

        if publisher_review and book_intro:
            combined_info = publisher_review + "\n" + book_intro
        elif publisher_review:
            combined_info = publisher_review
        elif book_intro:
            combined_info = book_intro
        else:
            combined_info = ""

        if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
            reason = "추천 이유 정보 없음"
        else:
            reason_prompt = (
                f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해라. "
                f"사용자 선호도에 기반해서 책의 특징이나 강점을 중심으로 설명해라. 만약 선호도를 모르겠다면, '추천 이유 정보 없음'을 출력해라.\n\n정보:\n{combined_info}"
            )
            reason_response = llm_clova.invoke(reason_prompt)
            generated_reason = reason_response.text().strip()
            if not generated_reason or len(generated_reason) < 10 or "추천 이유 정보 없음" in generated_reason:
                reason = "추천 이유 정보 없음"
            else:
                reason = generated_reason

        formatted = f"{title}\n{author}\n추천 이유: {reason}"
        formatted_answers.append(formatted)

    answer = "\n\n".join(formatted_answers)
    return answer, None

user_preferences = defaultdict(list)
log_history = []

In [58]:
def categorize_preference(question, response):
    if "장르" in question or "어떤 책" in question:
        user_preferences["genre"].append(response)
    elif "작가" in question or "좋아하는 작가" in question:
        user_preferences["author"].append(response)
    elif "목적" in question or "이유" in question:
        user_preferences["purpose"].append(response)
    else:
        user_preferences["misc"].append(response)

def robust_parse_llm_response(response_text):
    # 콜론(:)이랑 공백 파싱 방법
    score_match = re.search(r"검색\s*확률\s*[:：]\s*([\d\.]+)", response_text)
    search_score = float(score_match.group(1)) if score_match else None

    query_match = re.search(r"기본\s*검색\s*쿼리\s*[:：]\s*\"([^\"]+)\"", response_text)
    search_query = query_match.group(1).strip() if query_match else None

    follow_match = re.search(r"추가\s*질문\s*[:：]\s*\"([^\"]*)\"", response_text)
    follow_up_question = follow_match.group(1).strip() if follow_match else ""

    return search_score, search_query, follow_up_question

In [59]:
def search_and_generate_answer(query, query_history):
    while True:
        query_summary = "\n".join(query_history[-5:])
        prompt_vars = {
            "history": query_summary,
            "query": query,
            "impormation": "사용자 선호도 분석:",
            "strategy": "검색 전략:",
            "if": "검색이 가능한 경우:",
            "else": "추가 정보 필요:"
        }
        search_decision_dict = search_query_chain.invoke(prompt_vars)
        response_text = search_decision_dict["text"].strip()
        print("\n[🔍 LLM 응답 확인]\n", response_text)

        search_score, base_search_query, follow_up_question = robust_parse_llm_response(response_text)
        print(f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'")

        if search_score is None:
            print("\n[LLM 응답 파싱 실패: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

        if search_score >= 0.8 and base_search_query:
            final_search_query = final_query_chain.invoke({
                "history": "\n".join(query_history),
                "fallback": base_search_query
            })["text"].strip()
            print(f"\n[최종 검색 쿼리 생성]: {final_search_query}")

            answer, sources = generate_answer(final_search_query)
            if sources:
                book_info = "\n".join([f"- {title}" for title in sources])
                answer_with_info = f"{answer}\n\n[책 정보]\n{book_info}"
                print("\n[책 정보]\n", book_info)
            else:
                answer_with_info = answer
            return answer_with_info

        if follow_up_question:
            print(f"\n[보충 질문: {follow_up_question}]")
            query_history.append(f"AI: {follow_up_question}")
            user_response = input("\n사용자 응답: ")
            query_history.append(f"사용자: {user_response}")
            categorize_preference(follow_up_question, user_response)
            print("\n[사용자 선호도 업데이트 완료!]")
            query = f"{query} {follow_up_question} {user_response}"
            continue

        if search_score < 0.8 or not base_search_query:
            print("\n[검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

In [60]:
def interactive_multi_turn_qa():
    query_history = []

    while True:
        clear_output(wait=True)
        print("멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)

        query = input("질문을 입력하세요: ")

        if query.lower() == 'quit':
            print("\n[대화 저장 중...]")
            log_history.append(query_history)
            print("대화 저장 완료")
            break

        query_history.append(f"사용자: {query}")
        answer = search_and_generate_answer(query, query_history)

        print("\n[AI의 답변]")
        print(answer)

        query_history.append(f"AI: {answer}")

        input("\n-> 계속하려면 Enter를 누르세요...")

def show_log_history():
    print("\n[ 전체 대화 로그]")
    for i, session in enumerate(log_history, 1):
        print(f"\n 대화 세션 {i}:\n")
        print("\n".join(session))
        print("-" * 50)

In [61]:
# 실행
interactive_multi_turn_qa()

멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)
--------------------------------------------------
질문을 입력하세요: 조선시대 고전 문학을 추천해줘

[🔍 LLM 응답 확인]
 요약: 사용자가 조선시대 고전 문학을 추천해달라는 요청을 함

검색 확률 : 0.8
기본 검색 쿼리 : "조선시대 고전 문학"
추가 질문 : 어떤 장르를 원하시나요?

[디버그] 파싱 결과: 검색 확률=0.8, 기본 검색 쿼리='조선시대 고전 문학', 추가 질문=''

[최종 검색 쿼리 생성]: 1. 검색 확률: 0.8
2. 검색 쿼리: "조선시대 대표적인 고전 문학 작품"
3. 추가 질문 : 어떤 장르의 소설을 좋아하시나요? 
4. 추천 이유 : 조선시대는 다양한 고전 문학이 탄생한 시기로, 대표적인 작품으로는 춘향전, 홍길동전, 양반전 등이 있습니다. 하지만 개인의 취향에 따라 좋아하는 작품이 다를 수 있으므로, 원하시는 장르나 스타일을 알려주시면 더욱 정확한 추천을 해

[AI의 답변]
조선 왕실 이야기
공준원 지음
추천 이유: 이 책은 우리가 다 알고 있다고 생각하는 역사적 사건을 새로운 시각으로 재해석하여 흥미롭게 풀어낸다는 점에서 추천할 만합니다. 특히, 정사뿐만 아니라 비사까지 다루어 더욱 재미있게 읽을 수 있으며, 이를 통해 역사적 사건의 이면에 숨겨진 이야기를 알 수 있습니다. 또한, 각 장의 마지막에는 해당 사건에 대한 퀴즈를 수록하여 읽은 내용을 복습할 수 있도록 구성되어 있어 학습 효과도 높습니다.  이러한 요소들로 인해 이 책

청소년을 위한 고전 소설 에세이
류수열 지음
추천 이유: 이의 긴장이기도 하다. 이러한 긴장 속에서 작품을 읽는 일은 쉽지 않지만, 그만큼 큰 보람과 의미가 있다. 「허생전」부는 이런 점에서 청소년들에게 적극 추천할 만한 책이다. 

이 책은 조선 후기 대표적인 실학자 연암 박지원의 <허생전>을 비롯한 여러 고전 소설을 다루고 있다. 각 작품마다 작가의 생애와 시대적 배경, 작품의

KeyboardInterrupt: Interrupted by user

In [62]:
df[df['제목'].str.contains("조선 왕실 이야기", na=False)]

,ISBN,분류,제목,부제,저자,발행자,발행일,페이지,가격,표지,책소개,저자소개,목차,출판사리뷰,추천사
4933,9791170480440,['국내도서 > 역사 > 조선사 > 조선시대 일반'],조선 왕실 이야기,바로 알고 거울삼아야 할,공준원 지음,생각나눔,2020-06-04T15:00:00.000Z,416,15000,https://image.aladin.co.kr/product/24222/70/co...,정사와 비사를 오가는 흥미진진한 역사 이야기! 태조 이성계는 정말 차사를 죽였을까?...,공준원 1941년 김제시에서 태어난 그는 김제 중고등학교를 거쳐 고려대학교 법대를 ...,머리말\r\n \n이성계와 함흥차사 \n-이성계는 차사를 한 사람도 죽이지 않았다\...,제대로 알아두면 쓸모 있는 역사 역사는 무엇을 말하는가? 현대를 살아가는 우리에게 ...,NaN
